In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef
import torch
from transformers import TrainingArguments, Trainer
from transformers import MobileBertTokenizer, MobileBertForSequenceClassification
from transformers import EarlyStoppingCallback

### Load Data Set

In [2]:
df_train = pd.read_csv('training.csv')
df_val = pd.read_csv('validation.csv')
df_test = pd.read_csv('testing.csv')

df_train.head()

,Unnamed: 0,text,label
0,19429,retweet mentionhere: people be like from bae t...,1
1,10662,"i like being teased, do not give me the pussy ...",1
2,19558,retweet mentionhere: you are jealous bitch,1
3,2909,mentionhere cuz people at work are bitches the...,1
4,24378,quote this with your best racist comment i nee...,1


In [3]:
# Define pretrained tokenizer and model

model_name = 'google/mobilebert-uncased'

tokenizer = MobileBertTokenizer.from_pretrained(model_name)

model = MobileBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/847 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/147M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

In [4]:
# ----- 1. Preprocess data -----#
# Preprocess data

X_train = list(df_train["text"])
y_train = list(df_train["label"])

X_val = list(df_val["text"])
y_val = list(df_val["label"])

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [5]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [6]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [7]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average = 'weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average = 'weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, }

In [8]:
# Define Trainer

args = TrainingArguments(
    output_dir="D:\COS 802\Project\Mobilebert Training\output",
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps = 50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    seed=0,
    load_best_model_at_end=True,
    report_to="wandb",  # enable logging to W&B
    run_name="mobilebert"  # name of the W&B run (optional)
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [9]:
# Train pre-trained model
trainer.train()

***** Running training *****
  Num examples = 17348
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2170
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jesseparvess-elucidate (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1350483.200000,0.508235,0.810062,0.770028,0.810062,0.759331
100,0.623200,0.457303,0.871671,0.826713,0.871671,0.847937
150,0.401800,0.322654,0.887813,0.869543,0.887813,0.866630
200,0.356600,0.316346,0.893462,0.871388,0.893462,0.872703
250,0.389500,0.315876,0.894539,0.844046,0.894539,0.868528
300,0.927900,0.298990,0.894001,0.887523,0.894001,0.884020
350,0.320900,0.298988,0.895346,0.871796,0.895346,0.872252
400,0.352200,0.307311,0.894808,0.893237,0.894808,0.893353
450,0.384100,0.295987,0.899381,0.889875,0.899381,0.891486
500,0.298900,0.473119,0.903686,0.896214,0.903686,0.898864


***** Running Evaluation *****
  Num examples = 3717
  Batch size = 16
Saving model checkpoint to D:\COS 802\Project\Mobilebert Training\output\checkpoint-50
Configuration saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-50\config.json
Model weights saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-50\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3717
  Batch size = 16
C:\Users\jesse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to D:\COS 802\Project\Mobilebert Training\output\checkpoint-100
Configuration saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-100\config.json
Model weights saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-100\pytor

  Num examples = 3717
  Batch size = 16
Saving model checkpoint to D:\COS 802\Project\Mobilebert Training\output\checkpoint-1100
Configuration saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-1100\config.json
Model weights saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-1100\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3717
  Batch size = 16
Saving model checkpoint to D:\COS 802\Project\Mobilebert Training\output\checkpoint-1150
Configuration saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-1150\config.json
Model weights saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-1150\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3717
  Batch size = 16
Saving model checkpoint to D:\COS 802\Project\Mobilebert Training\output\checkpoint-1200
Configuration saved in D:\COS 802\Project\Mobilebert Training\output\checkpoint-1200\config.json
Model weights saved in D:\COS 802\Project\Mobilebert T

TrainOutput(global_step=2170, training_loss=31117.41712179206, metrics={'train_runtime': 14497.6552, 'train_samples_per_second': 2.393, 'train_steps_per_second': 0.15, 'total_flos': 293221634787792.0, 'train_loss': 31117.41712179206, 'epoch': 2.0})

In [10]:
# ----- 3. Predict -----#
# Load test data

X_test = list(df_test['text'])
y_test = list(df_test['label'])

X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)


In [11]:
# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

In [15]:
# Load trained model
model_path = "D:\COS 802\Project\Mobilebert Training\output\checkpoint-1500"
model = MobileBertForSequenceClassification.from_pretrained(model_path, num_labels=3)

loading configuration file D:\COS 802\Project\Mobilebert Training\output\checkpoint-1500\config.json
Model config MobileBertConfig {
  "_name_or_path": "google/mobilebert-uncased",
  "architectures": [
    "MobileBertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": false,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_feedforward_networks": 4,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "problem_type": "single_labe

In [16]:
# Define test trainer
test_trainer = Trainer(model)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

***** Running Prediction *****
  Num examples = 3718
  Batch size = 8


In [18]:
from sklearn.metrics import confusion_matrix, classification_report

In [19]:
confusion_matrix(y_test, y_pred)

array([[  76,  108,   29],
       [  74, 2756,   75],
       [   8,   37,  555]], dtype=int64)

In [20]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.48      0.36      0.41       213
           1       0.95      0.95      0.95      2905
           2       0.84      0.93      0.88       600

    accuracy                           0.91      3718
   macro avg       0.76      0.74      0.75      3718
weighted avg       0.91      0.91      0.91      3718

